In [2]:
import torch
import torch.nn as nn

# 設置隨機種子以確保結果可重現
torch.manual_seed(42)

# 定義輸入數據
# 輸入形狀: (batch_size, in_channels, sequence_length) = (1, 9, 3)
# 模擬通道 [0, 1, 2, 3, 4, 5, 6, 7, 8]
input_data = torch.zeros(1, 9, 3)
for i in range(9):
    input_data[0, i, :] = i  # 每個通道填充其索引值，例如通道 0 是 [0, 0, 0]，通道 1 是 [1, 1, 1]
# input_data[0] = tensor([[0, 0, 0], [1, 1, 1], [2, 2, 2], [3, 3, 3], ..., [8, 8, 8]])

# 定義 Conv1d 測試函數
def test_conv1d_groups(in_channels=9, out_channels=6, kernel_size=1, groups=3):
    print(f"\n=== Testing Conv1d with groups={groups} ===")
    
    # 檢查整除性
    if in_channels % groups != 0:
        raise ValueError(f"in_channels ({in_channels}) must be divisible by groups ({groups})")
    if out_channels % groups != 0:
        raise ValueError(f"out_channels ({out_channels}) must be divisible by groups ({groups})")
    
    # 創建 Conv1d 層
    conv = nn.Conv1d(in_channels, out_channels, kernel_size, groups=groups, bias=False)
    
    # 手動設置卷積核權重以便觀察
    with torch.no_grad():
        conv.weight.zero_()  # 先清零
        # 為每個組的卷積核設置可辨識的權重
        channels_per_group = in_channels // groups  # 每組 3 個通道
        out_channels_per_group = out_channels // groups  # 每組 2 個輸出通道
        for g in range(groups):
            for oc in range(out_channels_per_group):
                oc_idx = g * out_channels_per_group + oc
                for ic in range(channels_per_group):
                    ic_idx = g * channels_per_group + ic
                    conv.weight[oc_idx, ic, 0] = ic_idx + 1  # 權重為通道索引+1，例如通道 0 -> 1，通道 3 -> 4
    
    # 執行前向傳播
    output = conv(input_data)
    
    # 打印信息
    print(f"Input shape: {input_data.shape}")
    print(f"Output shape: {output.shape}")
    print(f"Weight shape: {conv.weight.shape}")
    print(f"Weights (conv kernel):\n{conv.weight[:, :, 0]}")
    print(f"Output:\n{output[0, :, :]}")
    
    return conv, output

# 測試 groups=3
try:
    conv, output = test_conv1d_groups(in_channels=9, out_channels=6, kernel_size=1, groups=3)
except ValueError as e:
    print(f"Error: {e}")


=== Testing Conv1d with groups=3 ===
Input shape: torch.Size([1, 9, 3])
Output shape: torch.Size([1, 6, 3])
Weight shape: torch.Size([6, 3, 1])
Weights (conv kernel):
tensor([[1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [4., 5., 6.],
        [7., 8., 9.],
        [7., 8., 9.]], grad_fn=<SelectBackward0>)
Output:
tensor([[  8.,   8.,   8.],
        [  8.,   8.,   8.],
        [ 62.,  62.,  62.],
        [ 62.,  62.,  62.],
        [170., 170., 170.],
        [170., 170., 170.]], grad_fn=<SliceBackward0>)
